In [1]:
import numpy as np
import pandas as pd
import json
import math

In [2]:
# Libraries that we cannot import for trading
import collections
import matplotlib.pyplot as plt
import re

import seaborn as sns
from IPython.display import display

# DATA LOADING

In [7]:
def split_data_by_symbol(df):
    markets = dict()
    df_grouped = df.groupby("symbol")
    symbols = list(df_grouped.groups.keys())
    for s in SYMBOLS:
        if s in df_grouped.groups:
            markets[s] = df_grouped.get_group(s).reset_index(drop=True)
    return markets

def load_hist_quote_data(files):
    md = [pd.read_csv(f,sep=';') for f in files]
    md = pd.concat(md, ignore_index=True)
    md.drop(columns=['profit_and_loss'], inplace=True)
    rename_columns = {'product':'symbol', 'bid_price_1': 'bid', 'ask_price_1': 'ask', 
                      'bid_volume_1':'bid_sz', 'ask_volume_1': 'ask_sz', 
                      'bid_price_2':'bid_2','bid_volume_2':'bid_sz_2','ask_price_2':'ask_2','ask_volume_2':'ask_sz_2',
                      'bid_price_3':'bid_3','bid_volume_3':'bid_sz_3','ask_price_3':'ask_3','ask_volume_3':'ask_sz_3'}
    md.rename(columns=rename_columns, inplace=True)
    for i in [4, 5]:
        for col in ['bid','ask']:
            md[f'{col}_{i}'] = np.nan
            md[f'{col}_sz_{i}'] = np.nan

    md['timestamp'] = md['timestamp'] + 1000000*md['day'] - 1000000*(md['day'].min())
    markets = split_data_by_symbol(md)
    symbols = list(markets.keys())
    return markets


def load_hist_trade_data(files, hist_days):
    md = [pd.read_csv(f,sep=';') for f in files]
    for i in range(len(hist_days)):
        md[i]['day'] = hist_days[i]
    md = pd.concat(md, ignore_index=True)
    md = md[md.price > 0].copy()
    md['timestamp'] = md['timestamp'] + 1000000*md['day'] - 1000000*(md['day'].min())
    
    markets = split_data_by_symbol(md)
    symbols = list(markets.keys())
    return markets

def augment_trade_data(trades):
    """ Making a guess to the sides of the trades based distance to mid price of previous quote
    Experimental - NOT SURE IF THIS IS GOING TO BE HELPFUL
    """
    bid_dist = trades['price'] - trades['bid']
    ask_dist = trades['ask'] - trades['price']
    trades['side'] = np.sign(bid_dist-ask_dist)
    return

def combine_hist_quote_trades(hist_quotes, hist_trades):
    """ Join data frames together
    """
    for sym in SYMBOLS:
        if not sym in hist_trades:
            continue
        quotes = hist_quotes[sym]
        trades = hist_trades[sym]
        # Join quote data to trades
        trades = trades.merge(quotes, how='left', on=['symbol','day','timestamp'])
        trades['notional'] = trades['price']*trades['quantity']

        augment_trade_data(trades)
        trades['buy_vol'] = trades.apply(lambda x: x['quantity'] if x['side'] == 1 else 0, axis=1)
        trades['sell_vol'] = trades.apply(lambda x: x['quantity'] if x['side'] == -1 else 0, axis=1)
        trades['grey_vol'] = trades.apply(lambda x: x['quantity'] if x['side'] == 0 else 0, axis=1)
        
        # Summarize state by state trade data
        trades_grp = trades.groupby(['symbol','day','timestamp']).agg({
            'quantity':'sum', 'notional':'sum', 'buy_vol':'sum', 'sell_vol':'sum','grey_vol':'sum'})
        
        trades_grp['vwap'] = trades_grp['notional']/trades_grp['quantity']
        trades_grp['trades'] = trades.groupby(['symbol','day','timestamp']).size()
        trades_grp = trades_grp.rename(columns={'quantity':'volume'})
        
        # Join trade data to quote
        quotes = quotes.merge(trades_grp.reset_index(), how='left')
        for col in ['volume', 'notional','trades', 'buy_vol','sell_vol','grey_vol']:
            quotes[col].fillna(0, inplace=True)
        quotes['vwap'].fillna(method='ffill',inplace=True)
        quotes.loc[quotes.vwap.isna(), 'vwap'] = quotes.loc[quotes.vwap.isna(), 'mid_price']
        
        # Update
        hist_quotes[sym] = quotes
        hist_trades[sym] = trades.copy()
    
    return hist_quotes, hist_trades

def load_hist_data(quote_files, trade_files, hist_days):
    hist_quotes = load_hist_quote_data(quote_files)
    hist_trades = load_hist_trade_data(trade_files, hist_days)
    
    hist_quotes, hist_trades = combine_hist_quote_trades(hist_quotes, hist_trades)
    return hist_quotes, hist_trades

In [8]:
def quote_levels_feature(quote):
    quote['bid_levels'] = 1
    quote['ask_levels'] = 1
    
    for i in range(2,6):
        quote.loc[quote[f'bid_{i}'].notna(), 'bid_levels'] += 1
        quote.loc[quote[f'ask_{i}'].notna(), 'ask_levels'] += 1
    
    return

def quote_weighted_px_full(quote, levels=5):
    notional = quote.bid_sz*quote.bid + quote.ask_sz*quote.ask
    size = quote.bid_sz + quote.ask_sz
    for i in range(2, min(quote['bid_levels'], levels)+1):
        notional += quote[f'bid_{i}']*quote[f'bid_sz_{i}']
        size += quote[f'bid_sz_{i}']
    for i in range(2, min(quote['ask_levels'], levels)+1):
        notional += quote[f'ask_{i}']*quote[f'ask_sz_{i}']
        size += quote[f'ask_sz_{i}']
    
    return notional/size


def quote_weighted_px_square(quote, levels=5):
    notional, size = 0, 0
    for x in ['bid','ask']:
        notional += np.square(quote[f'{x}_sz'])*quote[x]
        size += np.square(quote[f'{x}_sz'])
   
        for i in range(2, min(quote[f'{x}_levels'], levels)+1):
            notional += quote[f'{x}_{i}']*np.square(quote[f'{x}_sz_{i}'])
            size += np.square(quote[f'{x}_sz_{i}'])
    
    return notional/size

def quote_weighted_px_skip_top(quote, levels=5):
    notional, size = 0, 0
    
    if quote['bid_sz'] >= 5:
        notional += quote.bid_sz*quote.bid
        size += quote.bid_sz
    if quote['ask_sz'] >= 5:
        notional += quote.ask_sz*quote.ask
        size += quote.ask_sz

    for i in range(2, min(quote['bid_levels'], levels)+1):
        notional += quote[f'bid_{i}']*quote[f'bid_sz_{i}']
        size += quote[f'bid_sz_{i}']
    for i in range(2, min(quote['ask_levels'], levels)+1):
        notional += quote[f'ask_{i}']*quote[f'ask_sz_{i}']
        size += quote[f'ask_sz_{i}']
    
    return notional/size


MOVING_WINDOW_TIMES = [5, 10, 25, 50]
HIST_PX_TIMES = [1, 5]
FUTURE_TIMES = [1, 2, 3, 4, 5, 10, 25, 50, 100, 200, 500]

def quote_preprocess(md):
    """ Pre-computing additional columns for analyzing quote
    """
    # md['mid_price'] = (md.ask_price_1 + md.bid_price_1)/2
    if 'bid_levels' not in md.columns:
        quote_levels_feature(md)
    md['spread'] = md.ask - md.bid
    md['return'] = md['mid_price']/md['mid_price'].iloc[0] # % change in price since the beginning
    
#     for t in MOVING_WINDOW_TIMES:
#         md[f'mid_px_mavg{t}'] = md['mid_price'].rolling(t, min_periods=0).mean()
#         md[f'mid_px_mavg{t}_norm'] = md[f'mid_px_mavg{t}']/md['mid_price']
#         md[f'px_norm_mavg{t}'] = md['return'].rolling(t, min_periods=0).mean()
        
    # quote weighted average price
    quote_sz = md.bid_sz + md.ask_sz
    md['quote_wgt_px_top'] = (md.bid_sz*md.bid + md.ask_sz*md.ask)/quote_sz
    md['fair_v1'] = md.apply(quote_weighted_px_full, axis=1)
    md['fair_v2'] = md['vwap']
    md['fair_v3'] = md.apply(quote_weighted_px_square, axis=1)
    
    for t in HIST_PX_TIMES:
        # Price move since the past
        md[f'px_move_{t}'] = md.mid_price-md.mid_price.shift(t)
        md[f'bid_move_{t}'] = md.bid-md.bid.shift(t)
        md[f'ask_move_{t}'] = md.ask-md.ask.shift(t)
        
        md[f'px_move_{t}'].fillna(0, inplace=True)
        md[f'bid_move_{t}'].fillna(0, inplace=True)
        md[f'ask_move_{t}'].fillna(0, inplace=True)
    
    # Volume    
    for t in MOVING_WINDOW_TIMES:
        md[f'volume_{t}'] = md['volume'].rolling(t, min_periods=0).sum()
    if 'buy_vol' in md.columns:
        for t in MOVING_WINDOW_TIMES:
            md[f'buy_vol_{t}'] = md['buy_vol'].rolling(t, min_periods=0).sum()
            md[f'sell_vol_{t}'] = md['sell_vol'].rolling(t, min_periods=0).sum()
        md['volume_imb'] = md.buy_vol-md.sell_vol # Positive implies people are preparing for quote to move in
        
    md = md.copy()

    md['quote_imb'] = md.bid_sz/quote_sz
    md['levels_imb'] = md.bid_levels-md.ask_levels # Positive implies people are preparing for quote to move in
    
    ############# Future
    # Future prices
    for t in FUTURE_TIMES:
        # Future - present
        # Mid to mid, bid to mid, ask to mid
        md[f'fpx_{t}'] = md.mid_price.shift(-t).fillna(method='ffill') - md.mid_price
        md[f'fpx_bid_{t}'] = md.bid.shift(-t).fillna(method='ffill') - md.bid
        md[f'fpx_ask_{t}'] = md.ask.shift(-t).fillna(method='ffill') - md.ask
    for t in FUTURE_TIMES:
        md[f'fpx_move_{t}'] = np.sign(md[f'fpx_{t}'])
        md[f'fpx_bid_move_{t}'] = md[f'fpx_bid_{t}'] > 0
        md[f'fpx_ask_move_{t}'] = md[f'fpx_ask_{t}'] <= 0
    md['fvolume'] = md['volume'].shift(-1)
    for t in MOVING_WINDOW_TIMES:
        md[f'fvolume_{t}'] = md[f'volume_{t}'].shift(-t)
    md = md.copy()
        
    # Features
    md['total_bid_sz'] = md[['bid_sz','bid_sz_2','bid_sz_3']].sum(axis=1)
    md['max_bid_sz'] = md[['bid_sz','bid_sz_2','bid_sz_3']].max(axis=1)
    md['total_ask_sz'] = md[['ask_sz','ask_sz_2','ask_sz_3']].sum(axis=1)
    md['max_ask_sz'] = md[['ask_sz','ask_sz_2','ask_sz_3']].max(axis=1)
    
    md['bid_sz_mavg'] = md['bid_sz'].cumsum()/(1+md['timestamp']/100)
    md['ask_sz_mavg'] = md['ask_sz'].cumsum()/(1+md['timestamp']/100)
       
    for v in [1, 2, 3]:
        md[f'dist_fair_v{v}'] = md[f'fair_v{v}']-md['mid_price']
        md[f'dist_fair_v{v}_norm'] = md[f'dist_fair_v{v}']/md['mid_price']
        
        md[f'dist_fair_v{v}_bid'] = md[f'fair_v{v}']-md['bid'] # expected to be positive
        md[f'dist_fair_v{v}_ask'] = md[f'fair_v{v}']-md['ask'] # expected to be negative
            
    md['bid_improve'] = md['bid'] > md['bid'].shift(1)
    md['bid_revert'] = md['bid'] < md['bid'].shift(1)
    md['ask_improve'] = md['ask'] < md['ask'].shift(1)
    md['ask_revert'] = md['ask'] > md['ask'].shift(1)
    
    return md

def hist_data_summary(hist_quotes, hist_trades):
    for sym in SYMBOLS:
        print(sym)
        quote = hist_quotes[sym]
        quote_summary = quote.agg({
            'spread':['mean','std'], 'mid_price': ['mean','std'], 
            'fpx_move_10': ['mean','std'], 'fpx_move_1': ['mean','std'],
            'volume':['sum', 'mean'], 'trades':['sum', 'mean'],
            'bid_sz': 'mean', 'ask_sz':'mean', 
            'bid_levels':'mean', 'ask_levels':'mean',
            'bid_sz_2':'mean', 'ask_sz_2':'mean',
            'bid_sz_3':'mean', 'ask_sz_3':'mean',
        }).round(2)
        display(quote_summary)
        
fig_size = (6,3)

def plot_col_sym(md, sym, col):
    f, ax = plt.subplots(figsize=fig_size)
    ax.plot("timestamp", col, data=md)
    ax.set_xlabel("Time")
    ax.set_ylabel(col)
    ax.set_title(sym)
    plt.tight_layout()
    return

## Load

In [11]:
SYMBOLS = ['PEARLS','BANANAS']
SYMBOLS = ['COCONUTS','PINA_COLADAS']
SYMBOLS = ['DIVING_GEAR','BERRIES']
SYMBOLS = ['PICNIC_BASKET','DIP','BAGUETTE','UKULELE']

SYMBOLS = ['PEARLS','BANANAS', 'COCONUTS','PINA_COLADAS', 'DIVING_GEAR','BERRIES', 'PICNIC_BASKET','DIP','BAGUETTE','UKULELE']

In [13]:
quote_files, trade_files = [], []

data_dir = 'hist_data/island-data-bottle-round-1/'
hist_days = [-2]
quote_files += [f'{data_dir}prices_round_1_day_{i}.csv' for i in hist_days]
trade_files += [f'{data_dir}trades_round_1_day_{i}_wn.csv' for i in hist_days]

data_dir = 'hist_data/island-data-bottle-round-2/'
hist_days = [-1]
quote_files += [f'{data_dir}prices_round_2_day_{i}.csv' for i in hist_days]
trade_files += [f'{data_dir}trades_round_2_day_{i}_wn.csv' for i in hist_days]

data_dir = 'hist_data/island-data-bottle-round-3/'
hist_days = [0]
quote_files += [f'{data_dir}prices_round_3_day_{i}.csv' for i in hist_days]
trade_files += [f'{data_dir}trades_round_3_day_{i}_wn.csv' for i in hist_days]

data_dir = 'hist_data/island-data-bottle-round-4/'
hist_days = [1, 2, 3]
quote_files += [f'{data_dir}prices_round_4_day_{i}.csv' for i in hist_days]
trade_files += [f'{data_dir}trades_round_4_day_{i}_wn.csv' for i in hist_days]

hist_quotes, hist_trades = load_hist_data(quote_files, trade_files, [-2, -1, 0, 1, 2, 3])

In [79]:
trader_full_list = ['Peter','Mitch','Gary','Penelope','Omar',
               'Camilla','Caesar','Giulia','Mabel','Charlie',
               'Pablo','Olivia','Orson','Casey','George','Mya',
               'Max','Paris','Gina','Olga']

In [83]:
traders_set = set()
for sym in SYMBOLS:
    trades = hist_trades[sym]
    traders_set = traders_set.union(set(trades.seller.unique()))
    traders_set = traders_set.union(set(trades.buyer.unique()))
# N/A traders / traders that sit deeper in the book
print(set(trader_list).difference(traders_set))
print(traders_set)

{'Mitch', 'Mya', 'Giulia', 'Orson', 'George', 'Omar', 'Mabel', 'Casey', 'Max'}
{'Gary', 'Gina', 'Charlie', 'Camilla', 'Caesar', 'Olga', 'Paris', 'Peter', 'Pablo', 'Olivia', 'Penelope'}


In [95]:
# Trader summary
summary = []
for sym in SYMBOLS:
    quote = hist_quotes[sym]
    trades = hist_trades[sym].drop(columns=['bid_3','bid_sz_3','ask_3', 'ask_sz_3','bid_4', 'bid_sz_4', 'ask_4', 'ask_sz_4',
           'bid_5', 'bid_sz_5', 'ask_5', 'ask_sz_5'])
    close_px = quote.groupby('day')['mid_price'].last().rename('close_px')
    trades = trades.join(close_px, on='day')
    trades['spread'] = trades['ask']-trades['bid']

    trades_by_trader = {}
    for t in traders_set:
        buy_trades = trades.loc[trades.buyer == t].copy()
        buy_trades['trader_side'] = 1
        sell_trades = trades.loc[trades.seller == t].copy()
        sell_trades['trader_side'] = -1
        trades_by_trader[t] = pd.concat([buy_trades, sell_trades], ignore_index=True)

    for t in trades_by_trader:
        if trades_by_trader[t].empty:
            continue
        trades = trades_by_trader[t]
        trades['trade_pnl'] = trades['trader_side']*(trades['close_px']-trades['price'])

        # number of trades, pnl, avg size, avg spread, buy and sell
        trader_summary = {}
        trader_summary['trader'] = t
        trader_summary['sym'] = sym
        trader_summary['pnl'] = trades['trade_pnl'].sum()
        trader_summary['count'] = len(trades)
        trader_summary['buy_trades'] = len(trades.loc[trades.trader_side == 1])
        trader_summary['sell_trades'] = len(trades.loc[trades.trader_side == -1])
        trader_summary['volume'] = trades['quantity'].sum()
        trader_summary['buy_volume'] = trades.loc[trades.trader_side == 1]['quantity'].sum()    
        trader_summary['sell_volume'] = trades.loc[trades.trader_side == -1]['quantity'].sum()    
        trader_summary['avg_buy_size'] = trades.loc[trades.trader_side == 1]['quantity'].mean()
        trader_summary['avg_sell_size'] = trades.loc[trades.trader_side == -1]['quantity'].mean()
        trader_summary['avg_spread'] = trades['spread'].mean()
        trader_summary['buy_pnl'] = trades.loc[trades.trader_side == 1]['trade_pnl'].sum()
        trader_summary['sell_pnl'] = trades.loc[trades.trader_side == -1]['trade_pnl'].sum()    
        
        summary.append(trader_summary)

summary = pd.DataFrame.from_records(summary, index='trader')
summary

,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Gary,PEARLS,-4012.0,1174,592,582,5907,3047,2860,5.146959,4.914089,6.793867,-2520.0,-1492.0
Gina,PEARLS,-679.0,541,262,279,2252,1108,1144,4.229008,4.100358,2.863216,-546.0,-133.0
Charlie,PEARLS,20710.0,6965,3512,3453,17604,8809,8795,2.508257,2.547061,4.461019,7576.0,13134.0
Camilla,PEARLS,5615.0,2881,1495,1386,6211,3218,2993,2.152508,2.159452,5.927803,1725.0,3890.0
Caesar,PEARLS,13460.0,3621,1808,1813,5133,2559,2574,1.415376,1.419746,6.541011,5241.0,8219.0
Paris,PEARLS,-22326.0,8805,4426,4379,10171,5133,5038,1.159738,1.150491,5.074049,-14895.0,-7431.0
Pablo,PEARLS,-5270.0,2668,1303,1365,6889,3393,3496,2.603991,2.561172,3.302849,-3664.0,-1606.0
Penelope,PEARLS,-7498.0,2165,1012,1153,6423,3028,3395,2.992095,2.944493,6.799076,-4384.0,-3114.0
Gary,BANANAS,-2168.5,1163,558,605,13195,6484,6711,11.620072,11.092562,5.230439,-12694.0,10525.5


In [96]:
for t in traders_set:
    display(summary.loc[t])

,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Gary,PEARLS,-4012.0,1174,592,582,5907,3047,2860,5.146959,4.914089,6.793867,-2520.0,-1492.0
Gary,BANANAS,-2168.5,1163,558,605,13195,6484,6711,11.620072,11.092562,5.230439,-12694.0,10525.5
Gary,BERRIES,-2020.0,412,199,213,5865,2814,3051,14.140704,14.323944,6.368932,-8527.5,6507.5


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Gina,PEARLS,-679.0,541,262,279,2252,1108,1144,4.229008,4.100358,2.863216,-546.0,-133.0
Gina,BANANAS,-164.0,1034,490,544,6434,3204,3230,6.538776,5.937500,1.595745,-10335.0,10171.0
Gina,BERRIES,-101.5,33,14,19,215,103,112,7.357143,5.894737,3.000000,-377.0,275.5


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Charlie,PEARLS,20710.0,6965,3512,3453,17604,8809,8795,2.508257,2.547061,4.461019,7576.0,13134.0
Charlie,BANANAS,7392.0,11158,6084,5074,37858,19930,17928,3.275805,3.533307,3.996774,-110208.0,117600.0
Charlie,COCONUTS,4636.5,816,537,279,10019,7900,2119,14.711359,7.594982,2.650735,5811.5,-1175.0
Charlie,PINA_COLADAS,3006.0,594,397,197,3233,2535,698,6.385390,3.543147,2.909091,7321.5,-4315.5
Charlie,DIVING_GEAR,-14161.0,1329,800,529,2601,1576,1025,1.970000,1.937618,2.858540,25623.5,-39784.5
Charlie,BERRIES,199.0,58,22,36,626,258,368,11.727273,10.222222,6.810345,-667.0,866.0


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Camilla,PEARLS,5615.0,2881,1495,1386,6211,3218,2993,2.152508,2.159452,5.927803,1725.0,3890.0
Camilla,BANANAS,7166.5,2979,1437,1542,5888,2866,3022,1.994433,1.959792,4.845921,-26536.5,33703.0
Camilla,DIVING_GEAR,-18071.0,2359,1066,1293,4629,2074,2555,1.945591,1.976025,2.092836,61206.0,-79277.0
Camilla,BERRIES,10269.5,2087,1006,1081,18767,9176,9591,9.121272,8.872340,6.636320,-31422.0,41691.5
Camilla,PICNIC_BASKET,12297.5,720,427,293,3737,2110,1627,4.941452,5.552901,8.551389,15758.0,-3460.5
Camilla,DIP,17652.0,1017,26,991,7342,152,7190,5.846154,7.255298,1.501475,-778.0,18430.0
Camilla,BAGUETTE,-54464.5,546,6,540,3721,33,3688,5.500000,6.829630,1.430403,690.5,-55155.0
Camilla,UKULELE,45249.0,296,6,290,1976,14,1962,2.333333,6.765517,1.290541,-696.0,45945.0


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Caesar,PEARLS,13460.0,3621,1808,1813,5133,2559,2574,1.415376,1.419746,6.541011,5241.0,8219.0
Caesar,BANANAS,-3668.0,2221,1317,904,3358,1672,1686,1.269552,1.865044,4.514633,-23794.5,20126.5
Caesar,COCONUTS,73.0,1598,791,807,62922,31145,31777,39.374210,39.376704,1.453692,7880.0,-7807.0
Caesar,PINA_COLADAS,951.5,1121,610,511,18529,9708,8821,15.914754,17.262231,1.615522,5711.0,-4759.5
Caesar,PICNIC_BASKET,-11597.0,1053,563,490,1734,934,800,1.658970,1.632653,8.165242,5042.0,-16639.0
Caesar,DIP,-15946.5,3377,2186,1191,23862,15645,8217,7.156908,6.899244,1.647024,-41729.5,25783.0
Caesar,BAGUETTE,67559.5,2860,1720,1140,16392,10205,6187,5.933140,5.427193,1.570280,179345.0,-111785.5
Caesar,UKULELE,-37455.0,3016,1609,1407,10170,5966,4204,3.707893,2.987918,1.385942,-275878.5,238423.5


...

,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Paris,PEARLS,-22326.0,8805,4426,4379,10171,5133,5038,1.159738,1.150491,5.074049,-14895.0,-7431.0
Paris,BANANAS,-8077.5,11644,5384,6260,13319,6263,7056,1.163262,1.127157,4.254981,-121834.5,113757.0
Paris,DIVING_GEAR,32232.0,1956,956,1000,3864,1897,1967,1.984310,1.967000,2.455010,64529.0,-32297.0
Paris,BERRIES,-5720.5,1237,655,582,6753,3528,3225,5.386260,5.541237,6.449475,-24262.5,18542.0
Paris,DIP,-1705.5,2478,1224,1254,17196,8403,8793,6.865196,7.011962,1.675948,-26392.5,24687.0
Paris,BAGUETTE,-13095.0,2340,1147,1193,12815,6226,6589,5.428073,5.523051,1.596581,112679.5,-125774.5
Paris,UKULELE,-9045.0,2734,1408,1326,8228,4207,4021,2.987926,3.032428,1.393197,-239085.0,230040.0


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Peter,COCONUTS,-1199.5,708,358,350,5368,2713,2655,7.578212,7.585714,2.495763,1547.0,-2746.5
Peter,PINA_COLADAS,-2081.5,645,297,348,2206,1032,1174,3.474747,3.373563,2.613953,5841.5,-7923.0
Peter,BERRIES,534.5,133,59,74,391,171,220,2.898305,2.972973,2.992481,-1892.5,2427.0


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Pablo,PEARLS,-5270.0,2668,1303,1365,6889,3393,3496,2.603991,2.561172,3.302849,-3664.0,-1606.0
Pablo,BANANAS,2158.5,1913,821,1092,4765,1995,2770,2.429963,2.536630,2.117616,-18321.0,20479.5
Pablo,COCONUTS,-3510.0,408,79,329,7955,1374,6581,17.392405,20.003040,2.198529,256.0,-3766.0
Pablo,PINA_COLADAS,-1876.0,386,69,317,3888,653,3235,9.463768,10.205047,2.225389,1764.5,-3640.5
Pablo,BERRIES,-300.0,70,42,28,390,239,151,5.690476,5.392857,3.171429,-1220.0,920.0
Pablo,PICNIC_BASKET,7622.5,976,388,588,2774,1069,1705,2.755155,2.899660,6.050205,13215.5,-5593.0


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Olivia,BANANAS,512.0,17,8,9,204,102,102,12.750000,11.333333,5.117647,50.5,461.5
Olivia,BERRIES,506.0,12,6,6,136,68,68,11.333333,11.333333,5.916667,81.0,425.0
Olivia,UKULELE,1251.0,6,3,3,18,9,9,3.000000,3.000000,1.500000,139.5,1111.5


,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Penelope,PEARLS,-7498.0,2165,1012,1153,6423,3028,3395,2.992095,2.944493,6.799076,-4384.0,-3114.0
Penelope,BANANAS,-3116.0,1938,934,1004,10390,5185,5205,5.551392,5.184263,5.194014,-21558.5,18442.5
Penelope,BERRIES,-3367.0,638,337,301,7357,3893,3464,11.551929,11.508306,6.468652,-12449.5,9082.5
Penelope,PICNIC_BASKET,-8323.0,767,380,387,2681,1350,1331,3.552632,3.439276,11.311604,5103.5,-13426.5


In [97]:
summary.loc[summary.sym == 'DIVING_GEAR']

,sym,pnl,count,buy_trades,sell_trades,volume,buy_volume,sell_volume,avg_buy_size,avg_sell_size,avg_spread,buy_pnl,sell_pnl
trader,,,,,,,,,,,,,
Charlie,DIVING_GEAR,-14161.0,1329,800,529,2601,1576,1025,1.970000,1.937618,2.858540,25623.5,-39784.5
Camilla,DIVING_GEAR,-18071.0,2359,1066,1293,4629,2074,2555,1.945591,1.976025,2.092836,61206.0,-79277.0
Paris,DIVING_GEAR,32232.0,1956,956,1000,3864,1897,1967,1.984310,1.967000,2.455010,64529.0,-32297.0
